In [41]:
drkg_file = './data/v1_res/relation/DDires.csv'

In [42]:
from lib import format_transformer

In [43]:
import pandas as pd
import numpy as np

df = pd.read_csv(drkg_file)
df.head()

,Drug,Disease,Palliates_Hetionet,Treats_Hetionet,Effect_KEGG,Association_CTD
0,DrugBank:DB00997,DOID:363,0,1,0,1
1,DrugBank:DB00206,DOID:10763,0,1,0,1
2,DrugBank:DB00960,DOID:10763,0,1,0,1
3,DrugBank:DB00665,DOID:10283,0,1,1,1
4,DrugBank:DB00290,DOID:2998,0,1,0,1


In [44]:
trips = format_transformer.transform_kg_csv_to_triplets([drkg_file])

In [5]:
trips[0:5]

[['DrugBank:DB00997', 'Treats_Hetionet', 'DOID:363'],
 ['DrugBank:DB00997', 'Association_CTD', 'DOID:363'],
 ['DrugBank:DB00206', 'Treats_Hetionet', 'DOID:10763'],
 ['DrugBank:DB00206', 'Association_CTD', 'DOID:10763'],
 ['DrugBank:DB00960', 'Treats_Hetionet', 'DOID:10763']]

In [9]:
test_df.columns

Index(['Drug', 'Disease', 'Palliates_Hetionet', 'Treats_Hetionet',
       'Effect_KEGG', 'Association_CTD'],
      dtype='object')

In [ ]:
df = df.filter(['Drug', 'Palliates_Hetionet','Disease'], axis = 1)


In [5]:
df['Palliates_Hetionet'] = df['Palliates_Hetionet'].apply(str)

In [6]:
df.head()

,Drug,Palliates_Hetionet,Disease
0,DrugBank:DB00997,0,DOID:363
1,DrugBank:DB00206,0,DOID:10763
2,DrugBank:DB00960,0,DOID:10763
3,DrugBank:DB00665,0,DOID:10283
4,DrugBank:DB00290,0,DOID:2998


In [7]:
df = df[df.Palliates_Hetionet == '1']

In [8]:
triples = df.values.tolist()

In [9]:
triples[0]

['DrugBank:DB01175', '1', 'DOID:3312']

In [10]:
num_triples = len(triples)
num_triples

390

In [11]:
seed = np.arange(num_triples)
np.random.shuffle(seed)

train_cnt = int(num_triples * 0.9)
valid_cnt = int(num_triples * 0.05)
train_set = seed[:train_cnt]
train_set = train_set.tolist()
valid_set = seed[train_cnt:train_cnt+valid_cnt].tolist()
test_set = seed[train_cnt+valid_cnt:].tolist()

In [13]:
with open("train/drkg_train.tsv", 'w+') as f:
    for idx in train_set:
        f.writelines("{}\t{}\t{}\n".format(triples[idx][0], triples[idx][1], triples[idx][2]))
        
with open("train/drkg_valid.tsv", 'w+') as f:
    for idx in valid_set:
        f.writelines("{}\t{}\t{}\n".format(triples[idx][0], triples[idx][1], triples[idx][2]))

with open("train/drkg_test.tsv", 'w+') as f:
    for idx in test_set:
        f.writelines("{}\t{}\t{}\n".format(triples[idx][0], triples[idx][1], triples[idx][2]))

In [11]:
!DGLBACKEND=pytorch dglke_train --dataset WANGKG --data_path ./train \
                               --data_files drkg_train.tsv drkg_valid.tsv drkg_test.tsv \
                               --format 'raw_udd_hrt' --model_name TransE_l2 --batch_size 2048 \
                               --neg_sample_size 256 --hidden_dim 400 --gamma 12.0 --lr 0.1 \
                               --max_step 100000 --log_interval 1000 --batch_size_eval 16 -adv \
                               --regularization_coef 1.00E-07 --test --num_thread 1 \
                               --gpu 0 1 2 3 4 5 6 7 --num_proc 8 \
                               --neg_sample_size_eval 10000 --async_update

Using backend: pytorch
Traceback (most recent call last):
  File "/home/ubuntu/anthony/KG_LinkPrediction_Project/my_project_env/bin/dglke_train", line 8, in <module>
    sys.exit(main())
  File "/home/ubuntu/anthony/KG_LinkPrediction_Project/my_project_env/lib/python3.8/site-packages/dglke/train.py", line 81, in main
    dataset = get_dataset(args.data_path,
  File "/home/ubuntu/anthony/KG_LinkPrediction_Project/my_project_env/lib/python3.8/site-packages/dglke/dataloader/KGDataset.py", line 649, in get_dataset
    dataset = KGDatasetUDDRaw(data_path, data_name, delimiter, files, format, has_edge_importance)
  File "/home/ubuntu/anthony/KG_LinkPrediction_Project/my_project_env/lib/python3.8/site-packages/dglke/dataloader/KGDataset.py", line 415, in __init__
    assert os.path.exists(os.path.join(path, f)), \
AssertionError: File drkg_train.tsv not exist in ./train
